In [6]:
import numpy as np
import torch

import neural_network
from neural_network import sigmoid, sigmoid_der, ReLU, ReLU_der, LeakyReLU, LeakyReLU_der, identity, identity_der, mse, mse_der
import data_sets
import optimizers

In [7]:
# Adam, Runge

np.random.seed(1)
torch.manual_seed(1)

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [LeakyReLU, sigmoid, identity], [LeakyReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.Adam(0.01), 1_000)
    print(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(torch_net.parameters(), lr=0.01)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")


0.011290616565537563
0.010784314945340157
-----
0.006962843420107319
0.006406478118151426
-----
0.010204911388337764
0.00912395678460598
-----
0.007168155677114085
0.007427449803799391
-----
0.01134605192321906
0.011339052580296993
-----


In [12]:
# RMSprop, Runge

np.random.seed(5243)
torch.manual_seed(425)

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [ReLU, sigmoid, identity], [ReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.RMS_prop(0.0005), 1_000)
    print(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.RMSprop(torch_net.parameters(), lr=0.0005, alpha=0.9)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")


0.009110728639599004
0.0093979611992836
-----
0.009671294181236568
0.010088559240102768
-----
0.00965695459091457
0.011975628323853016
-----
0.015903443324095996
0.014931724406778812
-----
0.012476260804244885
0.015523722395300865
-----


In [ ]:
# Constant, Gaussian 2D

"""
lr=0.005
New best fit found: 0.3964916169643402 with seeds (1684542, 56171)
New best fit found: 0.33088116347789764 with seeds (1684543, 56172)
New best fit found: 0.3249157443642616 with seeds (1684550, 56179)
New best fit found: 0.3106874153017998 with seeds (1684554, 56183)
New best fit found: 0.3073898106813431 with seeds (1684556, 56185)
New best fit found: 0.30219993367791176 with seeds (1684567, 56196)
New best fit found: 0.2898058220744133 with seeds (1684572, 56201)


New best fit found: 0.38781340047717094 with seeds (426545624, 7814525) and learning rate 0.01
New best fit found: 0.34396616369485855 with seeds (426545624, 7814525) and learning rate 0.02
New best fit found: 0.31904836371541023 with seeds (426545624, 7814525) and learning rate 0.001
New best fit found: 0.3183331675827503 with seeds (426545626, 7814527) and learning rate 0.001
New best fit found: 0.30936188250780106 with seeds (426545627, 7814528) and learning rate 0.01
New best fit found: 0.3084338754415512 with seeds (426545628, 7814529) and learning rate 0.001


New best fit found: 0.23028772324323654 with seeds (426545629, 7814530) and learning rate 0.001, ss
"""

for i in range(100):

    np.random.seed(416185+i)
    torch.manual_seed(45136+i)

    for lr in [0.0005, 0.001, 0.002]:

        err = 0

        for _ in range(5):
            x_train, x_test, y_train, y_test = data_sets.load_gaussian2D_data(100)

            custom_net = neural_network.NeuralNetwork(2, [50, 50, 1], [sigmoid, sigmoid, identity], [sigmoid_der, sigmoid_der, identity_der], mse, mse_der)
            custom_net.train_network(x_train, y_train, 1, optimizers.Constant(lr), 1_000)
            # print(custom_net.cost(x_test, y_test))

            torch_net = torch.nn.Sequential(
                torch.nn.Linear(2, 50),
                torch.nn.Sigmoid(),
                torch.nn.Linear(50, 50),
                torch.nn.Sigmoid(),
                torch.nn.Linear(50, 1)
            )
            criterion = torch.nn.MSELoss()
            # torch.optim has no 'Constant' optimizer. Use SGD with a fixed learning rate instead.
            optimizer = torch.optim.SGD(torch_net.parameters(), lr=lr)
            for _ in range(1_000):
                inputs = torch.from_numpy(x_train.astype(np.float32))
                labels = torch.from_numpy(y_train.astype(np.float32))
                optimizer.zero_grad()
                outputs = torch_net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            torch_net.eval()
            with torch.no_grad():
                # print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
                err += np.norm(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))) - torch.from_numpy(y_test.astype(np.float32))).item())
                # print("-----")


0.15969910350617098
0.07623250037431717
-----
0.0948192246066435
0.07418946176767349
-----
0.20015568287808694
0.07532328367233276
-----
0.08722530391053754
0.06412351876497269
-----
0.1807445436634441
0.08512858301401138
-----
